In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np


In [2]:
import re
import string


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



In [4]:
import nltk


In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [6]:
import spacy
from transformers import pipeline

In [7]:
!pip install datasets


In [8]:
from datasets import load_dataset
import pandas as pd

# Load the dataset from Hugging Face
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Display the first few rows
df.head()


,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [9]:
!pip install spacy nltk



In [10]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:

nlp = spacy.load('en_core_web_sm')


In [12]:
!pip install swifter



In [13]:
import swifter
import nltk
import spacy
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK stopwords and tokenizer
nltk.download('stopwords')
nltk.download('punkt')

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Define the function BEFORE using it
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Handle non-string values safely

    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation

    tokens = word_tokenize(text)  # Tokenize
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords

    doc = nlp(' '.join(tokens))  # Process the text with spaCy
    return ' '.join([token.lemma_ for token in doc])  # Lemmatization

# Apply preprocessing using swifter
import swifter
df['processed_instruction'] = df['instruction'].swifter.apply(preprocess_text)

# Display processed data
df.head()


df['processed_instruction'] = df['instruction'].swifter.apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Pandas Apply:   0%|          | 0/26872 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/26872 [00:00<?, ?it/s]

In [14]:

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_instruction'])

# Target variable
y = df['intent']

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
# Initialize and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [17]:
# Predict on the test set
y_pred = model.predict(X_test)

# Display classification report
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

            cancel_order       1.00      0.96      0.98       187
            change_order       0.95      0.98      0.97       187
 change_shipping_address       0.99      0.99      0.99       216
  check_cancellation_fee       1.00      1.00      1.00       199
           check_invoice       0.97      0.99      0.98       192
   check_payment_methods       1.00      1.00      1.00       206
     check_refund_policy       1.00      1.00      1.00       200
               complaint       1.00      1.00      1.00       203
contact_customer_service       1.00      0.98      0.99       208
     contact_human_agent       0.98      1.00      0.99       201
          create_account       1.00      0.99      0.99       217
          delete_account       0.98      0.99      0.99       178
        delivery_options       0.99      1.00      1.00       218
         delivery_period       1.00      0.99      1.00       171
         

In [18]:
# Function to get response based on user query
def get_response(query):
    # Preprocess the query
    processed_query = preprocess_text(query)
    # Vectorize the query
    query_vec = vectorizer.transform([processed_query])
    # Predict intent
    intent = model.predict(query_vec)[0]
    # Retrieve the corresponding response
    response = df[df['intent'] == intent]['response'].values[0]
    return response

# Example usage
user_query = "How can I reset my password?"
print("User:", user_query)
print("Bot:", get_response(user_query))


User: How can I reset my password?
Bot: I'll get right on it! I can sense your eagerness to regain control over your user profile PIN. 

To reset your PIN, follow these simple steps:

1. Log in to your account on our platform.
2. Navigate to the account settings or profile settings section.
3. Look for the option to manage or change your PIN.
4. Click on the "Reset PIN" option.
5. Follow any additional instructions or verification steps provided on the screen.

If you encounter any difficulties or require further assistance during the process, don't hesitate to reach out. We're here to support you every step of the way!


In [19]:
# Initialize a transformer-based pipeline for generating responses
generator = pipeline('text-generation', model='gpt2')

# Function to generate a response using the transformer model
def generate_response(query):
    response = generator(query, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

# Example usage
user_query = "Tell me the process to change my shipping address."
print("User:", user_query)
print("Bot:", generate_response(user_query))


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: Tell me the process to change my shipping address.
Bot: Tell me the process to change my shipping address. Once I place you the "Packet #'s" on the delivery date that you wish to change your address to, you will receive a notification from the P2P program which you can update


In [20]:
# Note: This is a conceptual example. Rasa requires a specific project structure and command-line interface for training and running.

# Define a simple Rasa NLU model configuration
nlu_config = """
language: en
pipeline:
- name: WhitespaceTokenizer
- name: CountVectorsFeaturizer
- name: DIETClassifier
"""

# Save the configuration to a file
with open('config.yml', 'w') as file:
    file.write(nlu_config)

# Define training data
nlu_data = """
version: "2.0"
nlu:
- intent: greet
  examples: |
    - hi
    - hello
    - hey
- intent: goodbye
  examples: |
    - bye
    - goodbye
    - see you later
- intent: inform
  examples: |
    - I want to reset my password
    - How can I change my shipping address?
"""


 


import swifter

# Apply preprocessing with swifter for faster execution
df['processed_instruction'] = df['instruction'].swifter.apply(preprocess_text)

# Function to get AI response (Modify this to use your ML model)
def get_ai_response(query):
    processed_query = preprocess_text(query)
    
    # Check for an exact match in the dataset
    for index, row in df.iterrows():
        if processed_query in row['processed_instruction']:
            return row['response']  # Return the corresponding response from the dataset
    
    return "I'm sorry, I don't understand your query. Can you rephrase it?"

# Chatbot interaction loop
while True:
    user_query = input("Ask your question (or type 'exit' to stop): ")
    if user_query.lower() == 'exit':
        print("Chatbot session ended.")
        break
    response = get_ai_response(user_query)
    print(f"AI Response: {response}")

